In [81]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import sys
import numpy as np

pd.set_option('display.max_colwidth',None)
baseurl = 'https://cars.usnews.com'

In [88]:
driver = webdriver.Chrome("/usr/local/bin/chromedriver")

In [ ]:
def getcardata(cartype,carlink):
    f = open('car2.csv','w+')
    for i in range(len(carlink)):
        url = baseurl + carlink[i]
        carclass = cartype[i]
        print(f"staring for carclass {i+1} = {carclass}")
        iter = 1
        while iter <= 2:
            print("trying url:" + url)
            driver.get(url)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            all_matches = soup2.find_all('div', {'class': 'card shadow control'})
            for m in all_matches:
                try:
                    cname = m.find('a', {'class': 'vwo-rankings-profile-name'})
                    year,makemodel = cname.text.split(' ',maxsplit = 1)
                    for stat in m.find_all('div', {'class': 'rankings-stats__item'}):
                        key = stat.find('label', {'class': 'card__label'})
                        val = stat.find('span', {'class': 'rankings-stats__stat'})
                        if key.text.strip() == 'SCORE':
                            score = val.text.strip() 
                        if key.text.strip() == 'AVG PRICE PAID':
                            prange = val.text.strip().replace('$','').replace(',','').replace(' - ',' ').split()  
                            avgprice = (float(prange[0]) + float(prange[1]))/2 if len(prange) == 2 else ''
                        if key.text.strip() == 'MPG':
                            mpg = val.text.strip().replace(' / ',' ').split()  if val.text.strip() != 'N/A' else ''
                            mpgcity,_,mpghw,_ = mpg if len(mpg) == 4 else ['','','','']

                    print(f'{carclass},{makemodel},{score},{avgprice},{mpgcity},{mpghw}')  
                    f.write(f'{carclass},{makemodel},{score},{avgprice},{mpgcity},{mpghw}')
                    f.write('\n')
                except:
                    continue
            np = soup2.find('a', {'class' : 'pagination-link','alt' : 'Next Page'})
            if np is None:
                break
            if np.has_attr('href'):
                url = baseurl + np.attrs['href']        
            iter = iter + 1
            time.sleep(10)
        print("finishing for carclass = " + carclass)   
    f.close()

In [4]:
url = 'https://cars.usnews.com/cars-trucks/rankings/used'
# url = 'https://cars.usnews.com/cars-trucks/rankings/used/suvs'
# url = 'https://cars.usnews.com/cars-trucks/rankings/used/trucks'
# url = 'https://cars.usnews.com/cars-trucks/rankings/used/vans'
# url = 'https://cars.usnews.com/cars-trucks/rankings/used/hybrids'
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
all_matches = soup.find_all('section', {'class': 'rankings-type__section'})
ratings = all_matches[1]
categories = ratings.find_all('section',{'class' : 'small-12 medium-6 columns sub-type sub-type--year'})
cartype = []
carlink = []
for category in categories:
    cattype = category.find('h4', {'class': 'sub-type__name sub-type__name--used'})
    cartype.append(cattype.text)
    yearlink = category.find_all('a')
    carlink.append(yearlink[0].attrs['href'])
print(cartype)
print(carlink)
getcardata(cartype,carlink)

In [111]:
df = pd.read_csv('car_reviews.csv')
df.columns = ['CarClass', 'MakeModel', 'ReviewScore', 'AvgPaidPrice', 'MpgCity', 'MpgHwy']

In [112]:
df

,CarClass,MakeModel,ReviewScore,AvgPaidPrice,MpgCity,MpgHwy
0,AFFORDABLE SMALL CARS,HONDA FIT,8.7,16790.5,33.0,40.0
1,AFFORDABLE SMALL CARS,VOLKSWAGEN GTI,8.6,25174.5,25.0,31.0
2,AFFORDABLE SMALL CARS,CHEVROLET CRUZE,8.5,16985.5,28.0,38.0
3,AFFORDABLE SMALL CARS,KIA SOUL,8.5,22090.0,26.0,31.0
4,AFFORDABLE SMALL CARS,CHEVROLET SONIC,8.4,15104.0,26.0,34.0
...,...,...,...,...,...,...
492,LUXURY HYBRID AND ELECTRIC SUVS,AUDI E-TRON,8.0,60853.0,74.0,73.0
493,LUXURY HYBRID AND ELECTRIC SUVS,LEXUS NX HYBRID,7.9,NaN,33.0,30.0
494,LUXURY HYBRID AND ELECTRIC SUVS,ACURA MDX HYBRID,7.8,45580.5,26.0,27.0
495,LUXURY HYBRID AND ELECTRIC SUVS,LEXUS UX HYBRID,7.6,31759.0,41.0,38.0


In [113]:
df['IsLuxury'] = df['CarClass'].str.contains('LUXURY')

In [116]:
df['IsHybridOrElectric'] = df['CarClass'].str.contains('HYBRID') | df['CarClass'].str.contains('ELECTRIC')

In [119]:
df['IsSports'] = df['CarClass'].str.contains('SPORTS')  

In [121]:
df['CarClass'].unique()

array(['AFFORDABLE SMALL CARS', 'LUXURY SMALL CARS', 'HATCHBACKS',
       'COMPACT CARS', 'SUBCOMPACT CARS', 'AFFORDABLE MIDSIZE CARS',
       'LUXURY MIDSIZE CARS', 'AFFORDABLE LARGE CARS',
       'LUXURY LARGE CARS', 'AFFORDABLE SPORTS CARS',
       'LUXURY SPORTS CARS', 'AFFORDABLE CONVERTIBLES',
       'LUXURY CONVERTIBLES', 'WAGONS', 'HYBRID CARS',
       'LUXURY HYBRID CARS', 'AFFORDABLE COMPACT SUVS',
       'LUXURY COMPACT SUVS', 'AFFORDABLE SUBCOMPACT SUVS',
       'LUXURY SUBCOMPACT SUVS', 'AFFORDABLE MIDSIZE SUVS',
       'LUXURY MIDSIZE SUVS', 'AFFORDABLE LARGE SUVS',
       'LUXURY LARGE SUVS', 'HYBRID SUVS',
       'LUXURY HYBRID AND ELECTRIC SUVS', 'AFFORDABLE CROSSOVER SUVS',
       'AFFORDABLE SUVS WITH 2 ROWS', 'AFFORDABLE SUVS WITH 3 ROWS',
       'COMPACT TRUCKS', 'FULL SIZE TRUCKS', 'MINIVANS'], dtype=object)

In [123]:
df2 = df.drop_duplicates(subset=['MakeModel', 'ReviewScore', 'AvgPaidPrice', 'MpgCity', 'MpgHwy'])
df2.to_csv("car3.csv",index = False)